# GEOtiled Demo: A Scalable Workflow for Generating Large Datasets of High-Resolution Terrain Parameters

The GEOtiled workflow is comprised of three stages: 
1. Crop a preprocessed Digital Elevation Model (DEM) into tiles, each with a buffer region
2. Compute the desired terrain parameters for each individual tile concurrently
3. Mosaic each terrain parameter's tiles together

<p style="text-align:center">
    <img src="demo_images/geotiled_workflow.png" width="800"/>
</p>
<p style="text-align:center">
    <b>Figure 1. </b>GEOtiled Workflow
</p>
    
Terrain parameters are computed using DEMs from [USGS 3DEP Products](https://www.usgs.gov/3d-elevation-program/about-3dep-products-services) to compute over 15 topographic parameters. By default, this demo uses 3DEP products covering the entirety of the US state of Tennessee at a 30m resolution.

If you would like to work with a different region of data, go to the [USGS Data Download Application](https://apps.nationalmap.gov/downloader/#/elevation) and use the map to look for available DEM data. If using downloads links stored in a text file, said file should be stored in your working directory.

**IMPORTANT NOTE: Larger regions or higher resolutions will significantly increase the size of the data and the time to compute it.** 

## Environment Setup

The first cell below imports required libraries to run the notebook.

In [ ]:
# DO NOT MODIFY: Import library used to run notebook
import geotiled

### Settings

In the following cell you may specify variables such as what data to download, the size of the intermediary tiles to use for computation, and the file path where data will be stored. Comments for what each variable is for is included.

**Important Notes**
* For DEM download, three different methods are available: from a text file with a list of USGS download links, based off a shape file, or a specified latitude and longitude box
  * A text file from the USGS page should be stored in the working directory or child directory
  * Shape files are available for all US states and Washington DC. For the shapefile variable, specify the state abbreviation to use the correlating shapefile (e.g. TN for Tennessee)
  * A bounding box can be specified using the following syntax: {"xmin": val,"ymin": val,"xmax": val,"ymax": val}. X values correlate to longitude and Y values correlate to latitude (in degrees)

In [ ]:
# Data download variables
download_list = 'urls.txt'
dataset = '30m'
region = 'TN'

# Desired size of tiles
tile_size = [9137,4216]

# List of terrain parameters to compute
terrain_parameters = ['slope','aspect','hillshade']

# Specify the desired library to compute terrain parameters with
library = 'SAGA'

# Set the working directory
working_directory = '/media/volume/gabriel-geotiled/tn_30m'
geotiled.set_working_directory(working_directory)

## Preprocess the DEMs

### Fetch Data

`fetch_dem()` downloads DEMs directly from the USGS webpage within the bounds of a specified state shapefile or bounding box along with a desired resolution. The user has the option to either save the URLs with the download links for each DEM in a text file or download immediately. **Note that if both a shapefile and bounding box are given, the shapefile will take precedent.**

`download_files()` downloads the DEMs from a specified text file with download URLs or a Python list of strings containing the download URLs. If you would like to use your own text file to download DEMs, skip `fetch_dem()` and only run this function.

In [ ]:
# Create a text file with download URLs from a shape file
geotiled.fetch_dem(shapefile=region, txt_file=download_list, dataset=dataset)

In [ ]:
# Download files from the created text file
geotiled.download_files(download_list=download_list, download_folder='dem_tiles')

### Mosaic and Reproject DEMs

`build_mosaic()` merges together the downloaded DEM files into a single GeoTIFF file.

**Optional:** `reproject()` reprojects a specified GeoTIFF from its original coordinate system project to a new specified projection. It is important to ensure that the project has square grid cells for computational compatibility. DEMs from USGS are projected using the NAD83 (EPSG:4269) projection by default.

In [ ]:
# Build mosaic from DEMs
geotiled.build_mosaic(input_folder='dem_tiles', output_file='mosaic.tif', description='Elevation', cleanup=False)

In [ ]:
# Reproject the mosaic to Projected Coordinate System (PCS) EPSG:26918 - NAD83 UTM Zone 18N
geotiled.reproject(input_file='mosaic.tif', output_file='elevation.tif', projection='EPSG:26918', cleanup=False)

## Compute Terrain Parameters with GEOtiled

`crop_and_compute()` concurrently crops the preprocessed input DEM into tiles, computes desired terrain parameters, and crops off the buffer region of all computed terrain parameter tiles.

`build_mosaic()` is used to merge together the unbuffered terrain parameter tiles for each computed terrain parameter.

In [ ]:
# Run GEOtiled to crop and compute all terrain parameters for the given elevation data
geotiled.crop_and_compute(input_file='elevation.tif', column_length=tile_size[0], row_length=tile_size[1], parameter_list=terrain_parameters, compute_method=library)

In [ ]:
# Build mosaics for each of the computed parameter tiles
for terrain_parameter in terrain_parameters:
    geotiled.build_mosaic(input_folder=f"unbuffered_{terrain_parameter}_tiles", output_file=f"{terrain_parameter}.tif")

## Visualize the Results

`generate_img()` plots the GeoTIFF data. If you have large data, it may be beneficial to downsample the data for faster plotting. 

In [ ]:
# Visualize the terrain parameters
slp_img = geotiled.generate_img(tif='slope.tif', downsample=5, shp_files=region, crop_shp=True, reproject_gcs=True, title="TN Slope 30m", zunit="Radians", xyunit="Degree", ztype="Slope")
asp_img = geotiled.generate_img(tif='aspect.tif', downsample=5, shp_files=region, crop_shp=True, reproject_gcs=True, title="TN Aspect 30m", zunit="Radians", xyunit="Degree", ztype="Aspect")
hld_img = geotiled.generate_img(tif='hillshade.tif', downsample=5, shp_files=region, crop_shp=True, reproject_gcs=True, title="TN Hillshade 30m", zunit="Level", xyunit="Degree", ztype="Hillshade")

## End of Demo